In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import json

In [2]:
#function to count the number of parameters
def get_n_params(model): 
    np = 0
    for p in list(model.parameters()):
        np+= p.nelement()
    return np

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/malavika/anaconda3/envs/petitRT/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [4]:
pwd

'/home/malavika/Documents/Research/assessment/training'

In [5]:
path_to_file = "/home/malavika/Documents/Research/assessment/datasets/6params_train_bce_1M2.h5"
#path_to_file = "/home/malavika/Documents/Research/assessment/datasets/10params_train_bce_1M2.h5"

params = '6'

with h5py.File(path_to_file, 'r') as hdf:
    data = np.array(hdf.get('data')).astype(np.float32)
    label = np.array(hdf.get('labels')).astype(np.float32)

hdf.close()

In [6]:
np.shape(data)

(221852, 377)

In [7]:
# Normalize
a_min = np.min(data)
a_max = np.max(data)
data_norm = (data-a_min)/(a_max-a_min)

#Tensor
target = torch.from_numpy(label)
inpt = torch.from_numpy(data_norm)

class MyDataset(Dataset):
    def __init__(self):
        self.data = inpt
        self.target = target
        
    def __getitem__(self, index):
        x = self.data[index] 
        x = x[np.newaxis, ...] # To specify the number of channels c (here c=1)
        y = self.target[index]
        
        return {'input': x, 'target': y}
    
    def __len__(self):
        return len(self.data)

dataset = MyDataset()

In [8]:
split = [0.9, 0.1]
split_train = '0.9'
batch_size = 128 
indices = list(range(len(dataset)))
s = int(np.floor(split[1] * len(dataset)))

#shuffling
np.random.seed(111)
np.random.shuffle(indices)
train_indices, val_indices = indices[s:], indices[:s]

train_sampler, val_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, sampler=val_sampler)

In [9]:
len(train_dataloader)

1560

In [10]:
np.shape(train_dataloader.dataset.data)

torch.Size([221852, 377])

In [11]:
np.shape(train_dataloader.dataset.data)[1]
f = np.shape(train_dataloader.dataset.data)[1]

In [12]:
dataset_size = len(dataset)

In [13]:
dataloaders = {}
dataloaders['train'], dataloaders['val'] = train_dataloader, val_dataloader


In [14]:
len(dataloaders['train'])

1560

In [15]:
len(dataloaders['val'])

174

In [16]:
len(dataloaders['train'].dataset)

221852

In [17]:
# for batch_idx, sample in enumerate(dataloaders['train']):
#     print(len(dataloaders['train'].dataset))
#     print(len(dataloaders['train']))

In [18]:
#Models

class CNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 1, kernel_size=3)
        self.conv2 = nn.Conv1d(n_feature , n_feature, kernel_size=3)
        self.fc1 = nn.Linear(128*42, 128)
        self.fc2 = nn.Linear(128,1)
        
    def forward(self, x): 
        x = self.conv1(x)
        x = F.relu(x)
        x = max_pool1d(x,kernel_size=2)
        x = self.conv2(x)
        x = max_pool1d(x,kernel_size=2)
        x = self.conv2(x)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
    
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()        # Number of input features is 12.
        self.layer_1 = nn.Linear(f, 100) 
        self.layer_2 = nn.Linear(100, 32)
        self.layer_out = nn.Linear(32, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1)
        self.batchnorm2 = nn.BatchNorm1d(32)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        x = F.sigmoid(x)
        
        return x
    
    #layer, relu, bn, dp
    
class LinearModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
                           
    def get_weights(self):
        return self.weight
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = F.sigmoid(self.fc2(out)) #sigmoid as we use BCELoss
        return out

class Net(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        current_dim = input_dim
        self.layers = nn.ModuleList()
        for hdim in hidden_dim:
            self.layers.append(nn.Linear(current_dim, hdim))
            current_dim = hdim
        self.layers.append(nn.Linear(current_dim, output_dim))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = F.sigmoid(self.layers[-1](x))
        return out


In [19]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [20]:
#train

def train(n_epochs, model):
    
    best_loss = 0.0
    for epoch in range(n_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            
            #both    
            running_loss = 0.0
            
            for batch_idx, sample in enumerate(dataloaders[phase]):
                inputs = sample['input'].to(device)
                target = sample['target'].to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    output = model(inputs)
                    loss = criterion(torch.squeeze(output), torch.squeeze(target))
                    acc = binary_acc(torch.squeeze(output), torch.squeeze(target))
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   
                
                    running_loss += 1 * loss.item() * inputs.size(0) #loss for the phase/whole dataset

                if batch_idx % 100 == 0: 
                    print('{} epoch: {} [{}/{} ({:0.0f}%)]\tLoss: {:.6f}\tAcc: {:.2f}'.format(\
                    phase,epoch,(batch_idx+1)*len(sample['input']),len(dataloaders[phase].dataset),\
                    100.* ((batch_idx+1)*len(sample['input']))/len(dataloaders[phase].dataset),loss.item(), acc))
    
            if phase == 'train':
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[0]))
            else:
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[1]))

            if phase == 'val': 
                if epoch ==  (n_epochs-1) or running_loss < best_loss:
                    print('saving')
                    best_loss = running_loss
                    model_path = os.path.join(model_dir, 'model.pth')
                    torch.save(model.state_dict(), model_path)
                    
        with open(metrics_path, 'w') as f:
            json.dump(metrics, f, indent=4)
                    
#         print('--------------------------------------------------------------------')
        

In [21]:
#parameters

nb_epoch = 200
optimizer_name = 'Adam'
dl_arch = 'mlp'
metric = 'bce'
LEARNING_RATE = 0.001
criterion = nn.BCELoss()

model_path = '/home/malavika/Documents/Research/assessment/models/model1/'
model_name = 'model_'+params+'_spectra'+str(dataset_size)+'_split'+split_train+'_bs'+str(batch_size)+'_Loss'+\
            metric+'_opt'+str(optimizer_name)+'_lr'+str(LEARNING_RATE)+'_ep'+str(nb_epoch)+'_'+dl_arch

#hidden layers

hl2 = [128,32]
hl3 = [128,32,8]
hl4 = [128,64,32,8]
hl5 = [128,64,32,16,8]
hl6 = [128,64,32,16,8,4]

HL = [hl2, hl3, hl4, hl5, hl6] 


In [ ]:
for hl in HL:
    model_loop = Net(f, 1, hl)
    model_loop.to(device)
#     summary(model_loop, (1, f))
    optimizer = optim.Adam(model_loop.parameters(), lr=float(LEARNING_RATE))
    
    layers= str(len(hl))+'hl'
    model_dir = model_path + model_name+'_'+layers
    
    if not os.path.exists(model_dir):
            os.makedirs(model_dir)    
    metrics_path = os.path.join(model_dir, 'metrics.json')
    
    metrics = {
        'model': model_dir,
        'optimizer': optimizer.__class__.__name__,
        'criterion': criterion.__class__.__name__,
    #     'scheduler': scheduler.__class__.__name__,
        'dataset_size': int(len(dataset)),
        'train_size': int(split[0]*len(dataset)),
        'test_size': int(split[1]*len(dataset)),
        'n_epoch': nb_epoch,
        'batch_size': batch_size,
    #     'learning_rate': [],
        'train_loss': [],
        'val_loss': []
    }
    
    train(nb_epoch, model_loop)

/home/malavika/anaconda3/envs/petitRT/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train epoch: 0 [128/221852 (0%)]	Loss: 0.688611	Acc: 44.00
train epoch: 0 [12928/221852 (6%)]	Loss: 0.695077	Acc: 48.00
train epoch: 0 [25728/221852 (12%)]	Loss: 0.698434	Acc: 53.00
train epoch: 0 [38528/221852 (17%)]	Loss: 0.687644	Acc: 46.00
train epoch: 0 [51328/221852 (23%)]	Loss: 0.684887	Acc: 55.00
train epoch: 0 [64128/221852 (29%)]	Loss: 0.674369	Acc: 51.00
train epoch: 0 [76928/221852 (35%)]	Loss: 0.669688	Acc: 50.00
train epoch: 0 [89728/221852 (40%)]	Loss: 0.610727	Acc: 48.00
train epoch: 0 [102528/221852 (46%)]	Loss: 0.603302	Acc: 53.00
train epoch: 0 [115328/221852 (52%)]	Loss: 0.566783	Acc: 44.00
train epoch: 0 [128128/221852 (58%)]	Loss: 0.604892	Acc: 51.00
train epoch: 0 [140928/221852 (64%)]	Loss: 0.660942	Acc: 53.00
train epoch: 0 [153728/221852 (69%)]	Loss: 0.616082	Acc: 61.00
train epoch: 0 [166528/221852 (75%)]	Loss: 0.589299	Acc: 59.00
train epoch: 0 [179328/221852 (81%)]	Loss: 0.532808	Acc: 48.00
train epoch: 0 [192128/221852 (87%)]	Loss: 0.541599	Acc: 54.00
val 

train epoch: 7 [89728/221852 (40%)]	Loss: 0.333846	Acc: 69.00
train epoch: 7 [102528/221852 (46%)]	Loss: 0.350516	Acc: 53.00
train epoch: 7 [115328/221852 (52%)]	Loss: 0.293223	Acc: 59.00
train epoch: 7 [128128/221852 (58%)]	Loss: 0.313373	Acc: 55.00
train epoch: 7 [140928/221852 (64%)]	Loss: 0.366859	Acc: 62.00
train epoch: 7 [153728/221852 (69%)]	Loss: 0.433287	Acc: 56.00
train epoch: 7 [166528/221852 (75%)]	Loss: 0.395606	Acc: 53.00
train epoch: 7 [179328/221852 (81%)]	Loss: 0.324338	Acc: 64.00
train epoch: 7 [192128/221852 (87%)]	Loss: 0.424914	Acc: 62.00
val epoch: 7 [128/221852 (0%)]	Loss: 0.309230	Acc: 67.00
val epoch: 7 [12928/221852 (6%)]	Loss: 0.379163	Acc: 54.00
train epoch: 8 [128/221852 (0%)]	Loss: 0.343545	Acc: 54.00
train epoch: 8 [12928/221852 (6%)]	Loss: 0.347091	Acc: 56.00
train epoch: 8 [25728/221852 (12%)]	Loss: 0.301719	Acc: 62.00
train epoch: 8 [38528/221852 (17%)]	Loss: 0.437807	Acc: 55.00
train epoch: 8 [51328/221852 (23%)]	Loss: 0.354774	Acc: 55.00
train epoch:

train epoch: 14 [166528/221852 (75%)]	Loss: 0.354146	Acc: 61.00
train epoch: 14 [179328/221852 (81%)]	Loss: 0.425776	Acc: 68.00
train epoch: 14 [192128/221852 (87%)]	Loss: 0.352909	Acc: 72.00
val epoch: 14 [128/221852 (0%)]	Loss: 0.252507	Acc: 67.00
val epoch: 14 [12928/221852 (6%)]	Loss: 0.291830	Acc: 65.00
train epoch: 15 [128/221852 (0%)]	Loss: 0.351848	Acc: 62.00
train epoch: 15 [12928/221852 (6%)]	Loss: 0.373552	Acc: 55.00
train epoch: 15 [25728/221852 (12%)]	Loss: 0.257229	Acc: 62.00
train epoch: 15 [38528/221852 (17%)]	Loss: 0.366454	Acc: 64.00
train epoch: 15 [51328/221852 (23%)]	Loss: 0.326472	Acc: 73.00
train epoch: 15 [64128/221852 (29%)]	Loss: 0.251894	Acc: 71.00
train epoch: 15 [76928/221852 (35%)]	Loss: 0.362414	Acc: 66.00
train epoch: 15 [89728/221852 (40%)]	Loss: 0.370608	Acc: 65.00
train epoch: 15 [102528/221852 (46%)]	Loss: 0.342376	Acc: 62.00
train epoch: 15 [115328/221852 (52%)]	Loss: 0.268524	Acc: 72.00
train epoch: 15 [128128/221852 (58%)]	Loss: 0.281660	Acc: 64.0

train epoch: 22 [128/221852 (0%)]	Loss: 0.286997	Acc: 59.00
train epoch: 22 [12928/221852 (6%)]	Loss: 0.275799	Acc: 70.00
train epoch: 22 [25728/221852 (12%)]	Loss: 0.281840	Acc: 69.00
train epoch: 22 [38528/221852 (17%)]	Loss: 0.312668	Acc: 66.00
train epoch: 22 [51328/221852 (23%)]	Loss: 0.392489	Acc: 56.00
train epoch: 22 [64128/221852 (29%)]	Loss: 0.278413	Acc: 66.00
train epoch: 22 [76928/221852 (35%)]	Loss: 0.319114	Acc: 67.00
train epoch: 22 [89728/221852 (40%)]	Loss: 0.322312	Acc: 72.00
train epoch: 22 [102528/221852 (46%)]	Loss: 0.262178	Acc: 68.00
train epoch: 22 [115328/221852 (52%)]	Loss: 0.355386	Acc: 61.00
train epoch: 22 [128128/221852 (58%)]	Loss: 0.278586	Acc: 66.00
train epoch: 22 [140928/221852 (64%)]	Loss: 0.335917	Acc: 67.00
train epoch: 22 [153728/221852 (69%)]	Loss: 0.361243	Acc: 62.00
train epoch: 22 [166528/221852 (75%)]	Loss: 0.311283	Acc: 71.00
train epoch: 22 [179328/221852 (81%)]	Loss: 0.240839	Acc: 68.00
train epoch: 22 [192128/221852 (87%)]	Loss: 0.358391

...

train epoch: 80 [25728/221852 (12%)]	Loss: 0.206258	Acc: 80.00
train epoch: 80 [38528/221852 (17%)]	Loss: 0.239366	Acc: 82.00
train epoch: 80 [51328/221852 (23%)]	Loss: 0.319311	Acc: 77.00
train epoch: 80 [64128/221852 (29%)]	Loss: 0.152618	Acc: 82.00
train epoch: 80 [76928/221852 (35%)]	Loss: 0.225546	Acc: 79.00
train epoch: 80 [89728/221852 (40%)]	Loss: 0.193261	Acc: 77.00
train epoch: 80 [102528/221852 (46%)]	Loss: 0.277188	Acc: 80.00
train epoch: 80 [115328/221852 (52%)]	Loss: 0.165887	Acc: 77.00
train epoch: 80 [128128/221852 (58%)]	Loss: 0.138273	Acc: 72.00
train epoch: 80 [140928/221852 (64%)]	Loss: 0.243398	Acc: 77.00
train epoch: 80 [153728/221852 (69%)]	Loss: 0.235693	Acc: 83.00
train epoch: 80 [166528/221852 (75%)]	Loss: 0.226712	Acc: 87.00
train epoch: 80 [179328/221852 (81%)]	Loss: 0.145018	Acc: 80.00
train epoch: 80 [192128/221852 (87%)]	Loss: 0.159648	Acc: 77.00
val epoch: 80 [128/221852 (0%)]	Loss: 0.202731	Acc: 78.00
val epoch: 80 [12928/221852 (6%)]	Loss: 0.177303	Acc

train epoch: 87 [89728/221852 (40%)]	Loss: 0.233330	Acc: 75.00
train epoch: 87 [102528/221852 (46%)]	Loss: 0.248909	Acc: 86.00
train epoch: 87 [115328/221852 (52%)]	Loss: 0.255035	Acc: 73.00
train epoch: 87 [128128/221852 (58%)]	Loss: 0.217342	Acc: 82.00
train epoch: 87 [140928/221852 (64%)]	Loss: 0.122437	Acc: 84.00
train epoch: 87 [153728/221852 (69%)]	Loss: 0.170462	Acc: 87.00
train epoch: 87 [166528/221852 (75%)]	Loss: 0.267044	Acc: 74.00
train epoch: 87 [179328/221852 (81%)]	Loss: 0.188039	Acc: 83.00
train epoch: 87 [192128/221852 (87%)]	Loss: 0.272846	Acc: 80.00
val epoch: 87 [128/221852 (0%)]	Loss: 0.233750	Acc: 78.00
val epoch: 87 [12928/221852 (6%)]	Loss: 0.295681	Acc: 77.00
train epoch: 88 [128/221852 (0%)]	Loss: 0.308060	Acc: 81.00
train epoch: 88 [12928/221852 (6%)]	Loss: 0.182089	Acc: 80.00
train epoch: 88 [25728/221852 (12%)]	Loss: 0.216518	Acc: 72.00
train epoch: 88 [38528/221852 (17%)]	Loss: 0.192693	Acc: 85.00
train epoch: 88 [51328/221852 (23%)]	Loss: 0.215345	Acc: 82

train epoch: 94 [153728/221852 (69%)]	Loss: 0.136181	Acc: 84.00
train epoch: 94 [166528/221852 (75%)]	Loss: 0.266294	Acc: 75.00
train epoch: 94 [179328/221852 (81%)]	Loss: 0.223461	Acc: 72.00
train epoch: 94 [192128/221852 (87%)]	Loss: 0.335574	Acc: 78.00
val epoch: 94 [128/221852 (0%)]	Loss: 0.169759	Acc: 79.00
val epoch: 94 [12928/221852 (6%)]	Loss: 0.230061	Acc: 86.00
train epoch: 95 [128/221852 (0%)]	Loss: 0.149862	Acc: 88.00
train epoch: 95 [12928/221852 (6%)]	Loss: 0.195809	Acc: 84.00
train epoch: 95 [25728/221852 (12%)]	Loss: 0.137552	Acc: 88.00
train epoch: 95 [38528/221852 (17%)]	Loss: 0.214503	Acc: 77.00
train epoch: 95 [51328/221852 (23%)]	Loss: 0.152562	Acc: 84.00
train epoch: 95 [64128/221852 (29%)]	Loss: 0.149031	Acc: 80.00
train epoch: 95 [76928/221852 (35%)]	Loss: 0.196904	Acc: 77.00
train epoch: 95 [89728/221852 (40%)]	Loss: 0.200001	Acc: 88.00
train epoch: 95 [102528/221852 (46%)]	Loss: 0.238423	Acc: 83.00
train epoch: 95 [115328/221852 (52%)]	Loss: 0.144832	Acc: 81.0

val epoch: 101 [128/221852 (0%)]	Loss: 0.131488	Acc: 80.00
val epoch: 101 [12928/221852 (6%)]	Loss: 0.175112	Acc: 80.00
train epoch: 102 [128/221852 (0%)]	Loss: 0.166011	Acc: 80.00
train epoch: 102 [12928/221852 (6%)]	Loss: 0.231688	Acc: 79.00
train epoch: 102 [25728/221852 (12%)]	Loss: 0.322215	Acc: 77.00
train epoch: 102 [38528/221852 (17%)]	Loss: 0.273436	Acc: 76.00
train epoch: 102 [51328/221852 (23%)]	Loss: 0.132954	Acc: 90.00
train epoch: 102 [64128/221852 (29%)]	Loss: 0.220593	Acc: 80.00
train epoch: 102 [76928/221852 (35%)]	Loss: 0.207084	Acc: 84.00
train epoch: 102 [89728/221852 (40%)]	Loss: 0.160119	Acc: 81.00
train epoch: 102 [102528/221852 (46%)]	Loss: 0.163643	Acc: 84.00
train epoch: 102 [115328/221852 (52%)]	Loss: 0.253687	Acc: 80.00
train epoch: 102 [128128/221852 (58%)]	Loss: 0.188975	Acc: 80.00
train epoch: 102 [140928/221852 (64%)]	Loss: 0.263075	Acc: 74.00
train epoch: 102 [153728/221852 (69%)]	Loss: 0.242110	Acc: 80.00
train epoch: 102 [166528/221852 (75%)]	Loss: 0.

train epoch: 109 [12928/221852 (6%)]	Loss: 0.209624	Acc: 79.00
train epoch: 109 [25728/221852 (12%)]	Loss: 0.174547	Acc: 77.00
train epoch: 109 [38528/221852 (17%)]	Loss: 0.197113	Acc: 80.00
train epoch: 109 [51328/221852 (23%)]	Loss: 0.119034	Acc: 77.00
train epoch: 109 [64128/221852 (29%)]	Loss: 0.177104	Acc: 84.00
train epoch: 109 [76928/221852 (35%)]	Loss: 0.107653	Acc: 82.00
train epoch: 109 [89728/221852 (40%)]	Loss: 0.199941	Acc: 78.00
train epoch: 109 [102528/221852 (46%)]	Loss: 0.177340	Acc: 86.00
train epoch: 109 [115328/221852 (52%)]	Loss: 0.217388	Acc: 78.00
train epoch: 109 [128128/221852 (58%)]	Loss: 0.214069	Acc: 77.00
train epoch: 109 [140928/221852 (64%)]	Loss: 0.165984	Acc: 78.00
train epoch: 109 [153728/221852 (69%)]	Loss: 0.180802	Acc: 77.00
train epoch: 109 [166528/221852 (75%)]	Loss: 0.141549	Acc: 92.00
train epoch: 109 [179328/221852 (81%)]	Loss: 0.160936	Acc: 81.00
train epoch: 109 [192128/221852 (87%)]	Loss: 0.105296	Acc: 86.00
val epoch: 109 [128/221852 (0%)]	